In [1]:
import pandas as pd

base = pd.read_csv('dataset.csv')

In [2]:
import numpy as np

previsores = base.iloc[:1400000,0:20].values
previsores = np.reshape(previsores, (2000, 700, 20))
print(previsores.shape)

classes = base.iloc[:1400000, 28:31].values
classes = np.reshape(classes, (2000, 700, 3))
print(classes.shape)

(2000, 700, 20)
(2000, 700, 3)


In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, CuDNNGRU, Bidirectional, Masking

In [5]:
def criarRede():
    model = Sequential()
  
    #model.add(Masking(mask_value = 0, input_shape = (700, 20)))
  
    model.add(Bidirectional(CuDNNGRU(100, return_sequences = True), input_shape = (700, 20)))
    model.add(Dropout(0.3))
    
    model.add(Bidirectional(CuDNNGRU(100, return_sequences = True)))
    model.add(Dropout(0.3))

    model.add(Bidirectional(CuDNNGRU(100, return_sequences = True)))
    model.add(Dropout(0.3))
  
    model.add(Dense(3, activation = 'softmax'))
  
    model.compile(optimizer = 'adam', metrics = ['acc'], loss='categorical_crossentropy')
  
    return model

In [6]:
accu = []
precisions = []
recalls = []
f1 = []

In [7]:
from sklearn.metrics import balanced_accuracy_score, classification_report, precision_score, recall_score, f1_score, confusion_matrix

def train_and_evaluate_model(model, x_train, y_train, x_test, y_test):
    model.fit(x_train, y_train, epochs = 100, verbose = 1, batch_size = 32)
  
    predicted = model.predict(x_test)
  
    y_teste = []
    predict = []
  
    predicted = np.reshape(predicted, (predicted.shape[0] * predicted.shape[1], 3))
    y_test = np.reshape(y_test, (y_test.shape[0] * y_test.shape[1], 3))
    x_test = np.reshape(x_test, (x_test.shape[0] * x_test.shape[1], 20))

    for i in range(len(x_test)):
        cont = 0
        for j in range(len(x_test[i])):
            cont += x_test[i][j]
        if cont != 0:
            y_teste.append(y_test[i])
            predict.append(predicted[i])
    
    y_teste = np.asarray(y_teste)
    predict = np.asarray(predict)

    predicted = predict
    y_test = y_teste

    predicted = np.argmax(predicted, axis=1)
    y_test = np.argmax(y_test, axis=1)

    accu.append(balanced_accuracy_score(y_test, predicted)) 
    precisions.append(precision_score(y_test, predicted, average = 'weighted'))
    recalls.append(recall_score(y_test, predicted, average = 'weighted'))
    f1.append(f1_score(y_test, predicted, average = 'weighted'))
    
    print(classification_report(y_test, predicted))
    print('Acurácia')
    print(balanced_accuracy_score(y_test, predicted))
    print('Precisao')
    print(precision_score(y_test, predicted, average = 'weighted'))
    print('Recall')
    print(recall_score(y_test, predicted, average = 'weighted'))
    print('F1')
    print(f1_score(y_test, predicted, average = 'weighted'))
    print(confusion_matrix(y_test, predicted))

In [8]:
from sklearn.model_selection import KFold

kf = KFold(n_splits = 5, shuffle = True)

for train_index, test_index in kf.split(previsores):
    model = None
    model = criarRede()
    print("TRAIN:", train_index, "TEST:", test_index)
    train_and_evaluate_model(model, previsores[train_index], classes[train_index],
                           previsores[test_index], classes[test_index])

W0822 09:24:49.282961  9300 deprecation.py:506] From c:\users\gabriel\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0822 09:24:49.288946  9300 deprecation.py:506] From c:\users\gabriel\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\ops\init_ops.py:97: calling GlorotUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0822 09:24:49.289913  9300 deprecation.py:506] From c:\users\gabriel\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\ops\init_ops.py

TRAIN: [   0    2    3 ... 1996 1997 1999] TEST: [   1   10   12   13   32   35   36   40   53   54   58   62   69   76
  109  112  114  115  121  125  126  129  132  134  136  137  140  141
  142  147  158  159  165  167  172  175  178  182  190  194  206  214
  218  222  225  227  233  251  264  266  277  281  282  285  286  293
  294  298  299  304  312  313  344  347  352  356  361  363  364  367
  368  370  373  375  379  394  396  403  406  424  427  430  432  436
  449  450  457  460  461  462  464  477  485  487  494  497  498  500
  502  507  508  511  515  516  517  522  526  527  535  538  543  555
  560  568  569  571  572  580  581  584  593  601  603  606  612  613
  639  643  649  655  658  663  667  675  682  683  684  695  700  702
  714  715  717  718  730  731  733  745  748  760  762  763  764  766
  769  775  776  782  784  786  789  796  800  813  828  830  834  839
  840  842  846  850  864  870  882  885  888  896  901  908  921  922
  923  924  925  936  942  9

W0822 09:24:50.126676  9300 deprecation.py:323] From c:\users\gabriel\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/100
1600/1600 [==============================] - 10s 6ms/sample - loss: 0.3389 - acc: 0.8030
Epoch 2/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.3138 - acc: 0.8578
Epoch 3/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.3073 - acc: 0.8617
Epoch 4/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.3030 - acc: 0.8644
Epoch 5/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.3017 - acc: 0.8648
Epoch 6/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.2982 - acc: 0.8666
Epoch 7/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.2973 - acc: 0.8676
Epoch 8/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.2940 - acc: 0.8691
Epoch 9/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.2923 - acc: 0.8698
Epoch 10/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.2902 - acc: 0.871

1600/1600 [==============================] - 8s 5ms/sample - loss: 0.1601 - acc: 0.9313
Epoch 83/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.1591 - acc: 0.9317
Epoch 84/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.1587 - acc: 0.9318
Epoch 85/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.1575 - acc: 0.9325
Epoch 86/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.1570 - acc: 0.9333
Epoch 87/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.1552 - acc: 0.9339
Epoch 88/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.1552 - acc: 0.9336
Epoch 89/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.1539 - acc: 0.9340
Epoch 90/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.1533 - acc: 0.9345
Epoch 91/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.1530 - acc: 0.9342
Epo

...

Epoch 1/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.3393 - acc: 0.8387
Epoch 2/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.3163 - acc: 0.8566
Epoch 3/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.3108 - acc: 0.8602
Epoch 4/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.3048 - acc: 0.8636
Epoch 5/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.3033 - acc: 0.8645
Epoch 6/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.3012 - acc: 0.8653
Epoch 7/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.2979 - acc: 0.8667
Epoch 8/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.2950 - acc: 0.8686
Epoch 9/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.2941 - acc: 0.8686
Epoch 10/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.2919 - acc: 0.8700

1600/1600 [==============================] - 8s 5ms/sample - loss: 0.1611 - acc: 0.9308
Epoch 83/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.1599 - acc: 0.9310
Epoch 84/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.1594 - acc: 0.9311
Epoch 85/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.1602 - acc: 0.9314
Epoch 86/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.1588 - acc: 0.9320
Epoch 87/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.1573 - acc: 0.9325
Epoch 88/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.1556 - acc: 0.9333
Epoch 89/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.1551 - acc: 0.9324
Epoch 90/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.1535 - acc: 0.9322
Epoch 91/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.1547 - acc: 0.9321
Epo

1600/1600 [==============================] - 8s 5ms/sample - loss: 0.2269 - acc: 0.9028
Epoch 38/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.2251 - acc: 0.9035
Epoch 39/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.2229 - acc: 0.9048
Epoch 40/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.2217 - acc: 0.9052
Epoch 41/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.2197 - acc: 0.9062
Epoch 42/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.2187 - acc: 0.9066
Epoch 43/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.2174 - acc: 0.9070
Epoch 44/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.2143 - acc: 0.9089
Epoch 45/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.2123 - acc: 0.9094
Epoch 46/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.2104 - acc: 0.9102
Epo

Epoch 1/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.3351 - acc: 0.8320
Epoch 2/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.3110 - acc: 0.8591
Epoch 3/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.3034 - acc: 0.8638
Epoch 4/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.3002 - acc: 0.8656
Epoch 5/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.2978 - acc: 0.8669
Epoch 6/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.2971 - acc: 0.8670
Epoch 7/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.2935 - acc: 0.8684
Epoch 8/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.2913 - acc: 0.8696
Epoch 9/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.2881 - acc: 0.8711
Epoch 10/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.2852 - acc: 0.8733

1600/1600 [==============================] - 8s 5ms/sample - loss: 0.1579 - acc: 0.9332
Epoch 83/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.1572 - acc: 0.9338
Epoch 84/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.1565 - acc: 0.9338
Epoch 85/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.1552 - acc: 0.9349
Epoch 86/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.1546 - acc: 0.9347
Epoch 87/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.1536 - acc: 0.9355
Epoch 88/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.1528 - acc: 0.9355
Epoch 89/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.1530 - acc: 0.9358
Epoch 90/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.1522 - acc: 0.9361
Epoch 91/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.1502 - acc: 0.9370
Epo

In [9]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_12 (Bidirectio (None, 700, 200)          73200     
_________________________________________________________________
dropout_12 (Dropout)         (None, 700, 200)          0         
_________________________________________________________________
bidirectional_13 (Bidirectio (None, 700, 200)          181200    
_________________________________________________________________
dropout_13 (Dropout)         (None, 700, 200)          0         
_________________________________________________________________
bidirectional_14 (Bidirectio (None, 700, 200)          181200    
_________________________________________________________________
dropout_14 (Dropout)         (None, 700, 200)          0         
_________________________________________________________________
dense_4 (Dense)              (None, 700, 3)           

In [10]:
print('Acurácias total')
print(accu)
accu = np.array(accu)
print(accu.mean())
print('Precision total')
print(precisions)
precisions = np.array(precisions)
print(precisions.mean())
print('Recalls total')
print(recalls)
recalls = np.array(recalls)
print(recalls.mean())
print('F1 total')
print(f1)
f1 = np.array(f1)
print(f1.mean())

Acurácias total
[0.7108818622988616, 0.6944941986547039, 0.7042798908313416, 0.71071554907335, 0.7052419821911299]
0.7051226966098774
Precision total
[0.7278275997809438, 0.7110932565471653, 0.7218043947773508, 0.7250548551394168, 0.7175591509089533]
0.720667851430766
Recalls total
[0.7283055636753386, 0.7104569377292228, 0.7214455949209909, 0.725663122980914, 0.717661902340734]
0.7207066243294401
F1 total
[0.7266779039576208, 0.7095720255019374, 0.7201343010489973, 0.7247528601118983, 0.7172028772116273]
0.7196679935664162
